<a href="https://colab.research.google.com/github/mpfoster/Biochem5721/blob/master/09b_why_MM_5721.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Why not linearize Michaelis-Menten data?
## Biochemistry 5721 AU21

## Goals:
* recognize risks of interpreting linearized data
* learn python methods for data fitting
    * numpy.polyfit
    * scipy.curve_fit 
    * lmfit.model
    * residuals, error estimates

---
We'll start with an example from the literature: "Biochemical characterization of chitotriosidase enzyme: comparison between normal individuals and patients with Gaucher and with Niemann–Pick diseases", _Clinical Biochemistry_ **37**(10): 893-897. https://doi.org/10.1016/j.clinbiochem.2004.06.008

<img src="https://raw.githubusercontent.com/mpfoster/Biochem5721/master/images/bad-kms.png" width="250" align="right"/>In this publication, the authors characterize the steady-state behavior of the ezyme chitotriosidase, between healthy control (C) and diseased indiviuals. Figure 2, appears to show differences in the $K_M$ values for the enzyme from normal and diseased individuals. The graphed values were obtained from linear regression of Lineweaver-Burk plots of steady-state kinetics data. 

_An important lesson to learn from your science classes is that it is always a good idea to inspect and understand the experimental data, before drawing conclusions; that is arguably true in this case._

<img align="right" width="400" src="https://raw.githubusercontent.com/mpfoster/Biochem5721/master/images/l-b-normal.png"/>The raw experimental data (concentration v. time) are not shown, but the initial velocities as a function of substrate are shown in a Lineweaver-Burk plot in Figure 3. The data seem to show a reasonable linear regression of the data, revealing $x$- and $y$-intercepts, from which $K_M$ and $V_{max}$ values might be obtained. 

We will take a closer look. 

In [ ]:
# numerical and plotting libraries
import numpy as np
import matplotlib.pyplot as plt

# reciprocal values pulled from the figure:
r_S = np.array([145, 163, 180, 267, 401, 523, 837]) # / mM
r_v0 = np.array([0.016, 0.019, 0.021, 0.024, 0.031, 0.033, 0.04 ]) # nmol/h/mL = µM/h
plt.plot(r_S,r_v0, 'o')
plt.xlabel('$1/[s]_0\;/ mM^{-1}$',fontsize=16); plt.ylabel('$1/v_0$ /h/µM',fontsize=16)
plt.title('Wajner; Figure 3a data',fontsize=16)

Note that the Lineweaver-Burk plot should be **linear**; are these data?  Suitable steady-state kinetics data are fit to a straight line, for which the slope should be $K_m/V_{max}$, and $y$-intercept $1/V_{max}$

In [ ]:
# we'll use np.polyfit to fit the data with a 1st order polynomial y = mx+b
x = r_S; y = r_v0
popt,pcov=np.polyfit(x,y,1, cov=True) # m is slope; b is y-intercept
m = popt[0]; b = popt[1]
Vmax = 1/b; Km = m*Vmax 
print('Fitted Km: %.2g µM; Vmax: %.2g µM/h' % (Km,Vmax))
# let's choose a range of values to plot, from x intercept to past the last datapoint
last_x = np.max(x)*1.2
first_x = -b/m # with y = 0 = mx+b, x = -b/m
x_calc = np.array([first_x,last_x])
plt.plot(x_calc,m*x_calc+b)
plt.plot(x,y,'o')
plt.xlabel('$1/[s]_0$ x mM',fontsize=16); plt.ylabel('$1/v_0$',fontsize=16)
pars_LB = Vmax,Km
plt.title('$V_{max}$ = %.2f µM/h; $K_M$ = %.2g µM' % (pars_LB),fontsize=16)
plt.grid(); plt.show()

OK, this looks like the figure in the paper; the data are (kind of) approximated by a straight line. Let's have a closer look at how the data were actually recorded.

In [ ]:
# Let's plot the data as measured (not reciprocal)
S = 1/r_S; v0 = 1/r_v0
plt.plot(S,v0,'o')
plt.xlim(0,); plt.ylim(0,)
plt.xlabel('$[s]_0$ / mM',fontsize=16); plt.ylabel('$v_0$  / µM/h',fontsize=16)
plt.title('Wajner; Figure 3a data, inverted')
plt.grid(); plt.show()

### _Is this what we were expecting?_

Steady-state kinetic data for which the Lineweaver-Burk analysis is appropriate, should also match the Michaelis-Menten equation; this should resemble a hyperbola.

$$v_0 = \frac{V_{max}[S]}{[S] + K_M}$$ 

Let's use the fitted values from the Lineweaver-Burk analysis above to overlay a MM plot on the data.

In [ ]:
# MM eq:
def MM(x,Vmax,Km):
    return x*Vmax/(x+Km)
xval = np.linspace(0,np.max(S)) # we'll evaluate the function from 0 to max [S]
yval = MM(xval,*pars_LB) # (Vmax and Km values computed above)
plt.plot(S,v0,'o')
plt.plot(xval,yval)
plt.xlabel('$[s]_0$ /mM'); plt.ylabel('$v_0$  /µM/h')
plt.title('Wajner; Figure 3a data, MM eq; params from LB-fit')
plt.grid(); plt.show()

_Hmm. Despite the fact that the data basically look linear, the fit doesn't look terribly bad, if we ignore the point at high \[S\]._ Would we get the same values for $V_{max}$ and $K_M$ if we fit directly to the MM equation using non-linear least squares fitting?

---
## Why the difference? What is non-linear least-squares fitting anyway?

_Data fitting_ is a form of hypothesis testing, and a way of obtaining quantitative means of describing and predicting the outcome of an experiment. 

1. We start with a _prediction_ of some observable, and how that observable will change with changing experimental variables. In the cases above, we are posing that the measured enzyme catalyzed reaction will exhibit _hyperbolic_ Michaelis-Menten kinetics, with a half-maximal substrate concentration $K_M$ and maximal rate $V_{max}$. Moreover, that by measuring $v_0$ as a function of added $[S]$ will allow us to quantify differences between the activty of wild-type and disease-linked enzymes. So, the _experiment_ is just that: measure $v_0$ as a function of $[S]$. 
1. Having recorded the data, we then compare with our expectation. This comparison involves finding values of $K_M$ and $V_{max}$ that generate maximal agreement between the data and our model/predicted behavior.
1. The _agreement_ between data and model is evaluated by computing the _difference_ between the data and the model, using a parameter typically called the chi-square, $\chi^2$. For measurements (observations) over a series of $x$ values, $obs(x)$, we compute then square the difference (i.e., residual) between the observation and value predicted from the model, $calc(x)$: $$\chi^2(x) = \left(obs(x)-calc(x)\right)^2$$ For a given set of parameters, the $\chi^2$ for the model is the sum of the individual residuals (i.e., the _sum of squared residuals_, SSR). $$WSSR = \sum_{i}(w_i\chi_i)^2$$ The sum is _weighted_ by the uncertainty in each measurement ($w_i$), to account for variable accuracy in the measurements so that measurements with low precision have smaller weights.
1. The parameters of the model are then changed in a direction that _minimizes_ the $\chi^2$ (a.k.a WSSR). This _minimization_ is repeated until some _convergence criterium_ is reached. Those criteria are usually either (i) small residuals, or (ii) changing the parameters does not improve the fit. 
---
---

So, to start the fit, we need the _data_, $v_0$ vs. $[S]$, our _model_ $v_0 = V_{max}[S]/([S] + K_M)$, and some initial guesses for the parameters $V_{max}$ and $K_M$. _(For successful fitting, it is often the case that the choice of good starting parameters is critical for a successful minimization.)_ In the example above, we started with the Lineweaver-Burk parameters, but we could also try to make some guesses from the data itself. We see that the highest cocentration (~0.007 mM) has a $v_0 \approx 60$ µM/h, so we can make some informed starting guesses:

In [ ]:
# we enter som educated guesses for Km and Vmax from data plot
Km = ???
Vmax = ???
# now, let's compute our model (predicted v0) at each [S]
calc = MM(S,Vmax,Km)
# plot data and model together
plt.plot(S,v0,'o', label='data')
plt.plot(S,calc,'-', label='model')
plt.legend();plt.grid();plt.show()

Probably, our model and data will differ visually by a *lot*. The residual is computed by subtracting the model from the data: $\chi(x) = data(x)-model(x)$:

In [ ]:
# let's compute and plot the residuals: 
residual = (v0-calc)
plt.stem(S,residual,)
plt.xlabel('[S]'); plt.ylabel('Residuals'); plt.show()

Probably, the residuals will be large, and won't be randomly distributed around 0, and could be positive or negative. We square $chi$ so we don't have to worry about its sign and then vary the parameters to minimize the $\chi^2$, which for these parameters is:

In [ ]:
# Chisq 
chisq = (v0-calc)**2
SSR = np.sum(chisq)
print('Sum of squared residuals ("chi-square"): %.3f' % SSR)

### Minimization of $\chi^2$ by non-linear least squares optimization. 

The minimizaiton process is _non-linear_ because the model is not a straight line (for which a straightforward calculation tells your the best-fit parameters), and proceeds usually by computing in each cycle, the differential of $\chi^2$ with respect to the parameters being optimized. 

Here, we will use the `curve_fit` function from the `scipy` library:

In [ ]:
from scipy.optimize import curve_fit
'''
<function: scipy.optimize.minpack.curve_fit(f, xdata, ydata, p0=None, sigma=None, absolute_sigma=False, 
    check_finite=True, bounds=(-inf, inf), method=None, jac=None, **kwargs)>

for scipy's curve_fit function: we need to at least provide the function f, and x and y data.
we could provide initial parameters via p0
'''
myfunc = MM; x = S; y = v0
result = curve_fit(myfunc,x,y)
popt,pcov = result # curve_fit produces two arrays of parameters
# Always, always, look at the quality of the fit before the parameters:
yval = myfunc(xval,*popt) # we evaluate the function f at each xval 
plt.plot(xval,yval); plt.grid()
plt.plot(S,v0,'o')
print('Linear-fit of LB V_max: %.2f µM/h, K_M: %.2g mM' % (pars_LB[0],pars_LB[1]))
print('Direct-fit of MM V_max: %0.2f µM/h and K_M: %0.2g mM' % (popt[0],popt[1]))

OK, so we get _different_ best-fit parameters by directly fitting the data. Nevertheless, the agreement between data and model is not great. We again compute residuals and $\chi^2$:

In [ ]:
# we compute and plot the residuals: 
calc = MM(S,*popt)
residual = (v0-calc)
plt.stem(S,residual)
plt.xlabel('[S]'); plt.ylabel('Residuals'); plt.show()

It is **clear** that the data differ systematically from the model. This indicates either *problems with the data*, or that the *model is wrong*.

We also see that the parameters obtained from direct fit of the data differ significantly from those obtained from the double-reciprocal plot. A big reason for this is that the data don't seem to saturate, or have a shape of a hyperbola. This defect in the data is not so evident in the L-B plot.

_The obvious conclusion is that the kinetic data are not well described by the Michaelis-Menten equation, so parameters extracted from MM analysis are not trustworthy. That is harder to see in the Lineweaver-Burk plot than if the data are plotted ana analyzed directly._

### Standard error estimate
Still, let's give the authors the benefit of the doubt. Let's assume that the problem is just that the data are noisy, and that if repeated, a generally similar dataset would yield similar kinetic parameters. How can we evaluate the _precission_ of the fitted parameters? A simple means of estimating this, provided the measurement errors are uniform and symmetrically distributed around the best-fit values, is to compute from the variance and co-variance of the parameters; the covariance is a measure of how inter-related the two parameters are (when you change one, how $\chi^2$ depends on the other). When using `curve_fit` those parameters are stored in the second array, `pcov`.

In [ ]:
popt,pcov

In [ ]:
# standard errors in the best-fit parameters
print(f'Vmax = {popt[0]:0.2f} +/- {pcov[0,0]**0.5:0.2f} µM/h ({100*pcov[0,0]**0.5/popt[0]:0.1f}%)')
print('Km = %0.3g +/- %0.3g mM (%.1f%%)' % (popt[1],pcov[1,1]**0.5,100*pcov[1,1]**0.5/popt[1]))

_Obviously, if the uncertainty in the values we are comparing (e.g., $K_M$) are larger than the differences, we can't say much about those differences._

---
## lmfit
The operations we performed above using `scipy.curve_fit` are streamlined by using the `lmfit` functions. 
`mod = Model(MM)` generates the fitting object `mod` for the MM function. The `mod` object has many features, including a "fitter" that takes as input the y values, x values, and starting parameters for minimization, generating a new object with the `result`. Result has it's own features, including the best-fit parameters, plotting facilities, and statistical analysis of the data and fit:

In [ ]:
# if not lmfit not installed, uncomment the next line
!pip install lmfit
from lmfit import Model
mod = Model(MM)
result = mod.fit(v0,x=S,Km=1,Vmax=1)
result.plot(numpoints=100); plt.show()

In [ ]:
print(result.fit_report())

---
## Another example
[Mann _et al._, _Mol Cell_ 2003 **12**(4):925-35.](https://linkinghub.elsevier.com/retrieve/pii/S1097-2765(03)00357-5) "Eukaryotic RNase P: Role of RNA and Protein Subunits of a Primordial Catalytic Ribonucleoprotein in RNA-Based Catalysis"

In this paper, the authors sought to compare the function of one of the protein subunits from human RNase P (Rpp29) with the bacterial protein C5, by comparing their effect on the steady-state kinetics of the enzyme (which cleaves precursor tRNA molecules). 
<img width="600" align="right" src="https://marlin-prod.literatumonline.com/cms/attachment/a62da6f6-154c-48ac-a305-1894d9abffdb/gr6.gif"/>Figure 6, panels B and C show double-reciprocal Lineweaver-Burk plots of the kinetic data. The data were fit with a line, and reported $V_{max}$ and $K_m$ values are shown.

Closer inspection of the plots _should_ make us circumspect about the meaning of those parameters. Note, for example, that at high 1/\[S\], both datasets deviate from the line, and in particular, for the Rpp29 dataset, the three values at low 1/\[S\] have the _same_ $1/V$ ?! 

Let's examine these data:

In [ ]:
recip_c = np.array([8,10,14,20,40,80])
p29_rV = np.array([55,55,55,125,167,200])
c5_rV = np.array([8,10,13,18,24,40])
c=1/recip_c; p29_V = 1/p29_rV; c5_V = 1/c5_rV # compute c and v0 from 1/x
plt.figure(figsize=(10, 4))
plt.subplot(121); plt.plot(c,c5_V,'o-')
plt.ylabel('V / pmol/min'); plt.xlabel('[tRNA] / pmol/μL'); plt.title('C5',size=18)
plt.subplot(122); plt.plot(c,p29_V,'o-')
plt.xlabel('[tRNA] / pmol/μL'); plt.title('Rpp29', size=18)
plt.show()

So, what do we notice?

Well...
1. for the C5 data, V doesn't seem to saturate; this suggests that $K_M$ is well beyond the maximum tested substrate concentration ($\gg 0.12$ pmol/μL), and that $V_{max}$ is unknown.
2. As noted above, for Rpp29 the three highest concentrations of substrate, V was the same, which if correct, would invite the conclusion that the enzyme has been saturated, an $K_M$ is *much* lower than for Rpp29, and well below 0.08 pmol/μL. 

We **can** fit the data with the MM equation, but becuase they do not seem hyperbolic, we should be skeptical of the resulting parameters:

In [ ]:
# here, we'll use scipy curve_fit, instead of lmfit
from scipy.optimize import curve_fit
'''
<function: scipy.optimize.minpack.curve_fit(f, xdata, ydata, p0=None, sigma=None, absolute_sigma=False, 
    check_finite=True, bounds=(-inf, inf), method=None, jac=None, **kwargs)>
'''
# first the C5 data
f = MM; x = c; y = c5_V
c5_result = curve_fit(f,x,y)
popt,pcov = c5_result
print('Best-fit V_max: %0.3f pmol/min and K_M: %0.3f pmol/µL' % (popt[0],popt[1]) )

In [ ]:
# now the p29 data
f = MM; x = c; y = p29_V
p29_result = curve_fit(f,x,y)
popt,pcov = p29_result
print('Best-fit V_max: %0.3f pmol/min and K_M: %0.3f pmol/µL' % (popt[0],popt[1]) )

In [ ]:
# plot the data
c_eval = np.linspace(0,np.max(c))
plt.figure(figsize=(10, 4))
plt.subplot(121); plt.plot(c,c5_V,'o')
popt,pcov = c5_result; plt.plot(c_eval,MM(c_eval,*popt), label='Vmax, Km: %s' % popt);
plt.ylabel('V /pmol/min'); plt.xlabel('[tRNA] /pmol/μL'); plt.title('C5',size=18)
plt.legend(); plt.grid()
plt.subplot(122); plt.plot(c,p29_V,'o-')
popt,pcov = p29_result; plt.plot(c_eval,MM(c_eval,*popt), label='Vmax, Km: %s' % popt);
plt.xlabel('[tRNA] /pmol/μL'); plt.title('Rpp29', size=18); plt.legend()
plt.grid(); plt.show()

Those parameters differ, but not wildly, from the L-B fitted parameters, but can we trust them?

When scipy (directly or via lmfit), a variance and covariation matrix is computed `pcov`, above. This covariation matrix describes the interdependence of the fitted parameters and allows us to compute an _estimate_ for the certainty in the fitted parameters:

In [ ]:
popt,pcov = c5_result
print('C5 Vmax = %0.3f +/- %0.3f pmol/min (%.0f%%)' % (
    popt[0],pcov[0,0]**0.5, 100*pcov[0,0]**0.5/popt[0]))
print('C5 Km = {:.3f} +/- {:.3f} pmol/min ({:.0f}%)'.format(
    popt[1],pcov[1,1]**0.5,100*pcov[1,1]**0.5/popt[1]))
# (two different ways to format print statements)
popt,pcov = p29_result
print('Rpp29 Vmax = %0.3f +/- %0.3f pmol/min (%0.0f%%)' % (popt[0],pcov[0,0]**0.5, 
                                                           100*pcov[0,0]**0.5/popt[0]))
print('Rpp29 Km = {:.3f} +/- {:.3f} pmol/min ({:.0f}%)'.format(
    popt[1],pcov[1,1]**0.5,
    100*pcov[1,1]**0.5/popt[1]))

We can see that the computed standard errors are large (~38-85%) of the best-fit values, but even then they are likely to be under-estimated.